<a href="https://colab.research.google.com/github/amitadhainje/Coursera-Intro-to-TF-for-AI-ML-DP/blob/master/Coursera_Natural_Language_Processing_WithTF_Week_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
             'I love badminton',
             'you love dogs',
             'Do you think the dog is amazing ?'
]

tokenizer = Tokenizer(num_words=100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

paddedSentences = pad_sequences(sequences, padding="post")

print (word_index)
print ("\n")
print (sequences)
print ("\n")
print (paddedSentences)

{'<OOV>': 1, 'love': 2, 'you': 3, 'i': 4, 'badminton': 5, 'dogs': 6, 'do': 7, 'think': 8, 'the': 9, 'dog': 10, 'is': 11, 'amazing': 12}


[[4, 2, 5], [3, 2, 6], [7, 3, 8, 9, 10, 11, 12]]


[[ 4  2  5  0  0  0  0]
 [ 3  2  6  0  0  0  0]
 [ 7  3  8  9 10 11 12]]


In [2]:
import tensorflow as tf
print (tf.__version__)

2.2.0-rc2


In [3]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info="True", as_supervised="True")
print (info)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIPKUW6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIPKUW6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIPKUW6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew

In [4]:
import numpy as np

train_data , test_data = imdb['train'], imdb['test']

trainingSentences = []
trainingLabels = []

for s,l in train_data:
    trainingSentences.append(str(s.numpy()))
    trainingLabels.append(l.numpy())


testSentences = []
testLabels = []

for s,l in test_data:
    testSentences.append(str(s.numpy()))
    testLabels.append(l.numpy())

print (trainingSentences[0])



b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."


In [0]:
trainLabels = np.array(trainingLabels)
testLabels = np.array(testLabels)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length =120
trunc_type = "post"
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(trainingSentences)
word_index = tokenizer.word_index

trainSeq = tokenizer.texts_to_sequences(trainingSentences)
paddedTrainSeq = pad_sequences(trainSeq, padding="post", maxlen=max_length)

testSeq = tokenizer.texts_to_sequences(testSentences)
paddedTestSeq = pad_sequences(testSeq, padding="post", maxlen=max_length)

In [21]:
paddedTrainSeq[0]

array([  59,   12,   14,   35,  439,  400,   18,  174,   29,    1,    9,
         33, 1378, 3401,   42,  496,    1,  197,   25,   88,  156,   19,
         12,  211,  340,   29,   70,  248,  213,    9,  486,   62,   70,
         88,  116,   99,   24, 5740,   12, 3317,  657,  777,   12,   18,
          7,   35,  406, 8228,  178, 2477,  426,    2,   92, 1253,  140,
         72,  149,   55,    2,    1, 7525,   72,  229,   70, 2962,   16,
          1, 2880,    1,    1, 1506, 4998,    3,   40, 3947,  119, 1608,
         17, 3401,   14,  163,   19,    4, 1253,  927, 7986,    9,    4,
         18,   13,   14, 4200,    5,  102,  148, 1237,   11,  240,  692,
         13,   44,   25,  101,   39,   12, 7232,    1,   39, 1378,    1,
         52,  409,   11,   99, 1214,  874,  145,   10,    0,    0],
      dtype=int32)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, GlobalAveragePooling1D

model1 = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Flatten(),
        #tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

model1.summary()

model1.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 11526     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [29]:
model1.fit(paddedTrainSeq, trainLabels, epochs=5, validation_data=(paddedTestSeq, testLabels))

Epoch 1/5
782/782 [==============================] - 4s 5ms/step - loss: 0.0024 - accuracy: 0.9999 - val_loss: 0.6121 - val_accuracy: 0.8396
Epoch 2/5
782/782 [==============================] - 4s 5ms/step - loss: 9.7025e-04 - accuracy: 1.0000 - val_loss: 0.6536 - val_accuracy: 0.8408
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 4.8849e-04 - accuracy: 1.0000 - val_loss: 0.6926 - val_accuracy: 0.8404
Epoch 4/5
782/782 [==============================] - 4s 5ms/step - loss: 2.8329e-04 - accuracy: 1.0000 - val_loss: 0.7298 - val_accuracy: 0.8404
Epoch 5/5
782/782 [==============================] - 4s 5ms/step - loss: 1.6759e-04 - accuracy: 1.0000 - val_loss: 0.7658 - val_accuracy: 0.8406


In [30]:
model2 = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        #tf.keras.layers.Flatten(),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

model2.summary()

model2.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])



Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


Check the model summary changes by using the "flatten" and "GlobalAveragePooling1D" layer. "GlobalAveragePooling1D" reduces the dimensions drastically and hence makes the model much faster

In [31]:
model2.fit(paddedTrainSeq, trainLabels, epochs=5, validation_data=(paddedTestSeq, testLabels))

Epoch 1/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5485 - accuracy: 0.7665 - val_loss: 0.3863 - val_accuracy: 0.8423
Epoch 2/5
782/782 [==============================] - 4s 5ms/step - loss: 0.3147 - accuracy: 0.8738 - val_loss: 0.3256 - val_accuracy: 0.8589
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 0.2529 - accuracy: 0.8996 - val_loss: 0.3168 - val_accuracy: 0.8618
Epoch 4/5
782/782 [==============================] - 4s 5ms/step - loss: 0.2195 - accuracy: 0.9143 - val_loss: 0.3246 - val_accuracy: 0.8599
Epoch 5/5
782/782 [==============================] - 4s 5ms/step - loss: 0.1952 - accuracy: 0.9273 - val_loss: 0.3399 - val_accuracy: 0.8552
